## Imports

In [1]:
import pandas as pd 
from glob import glob
from pathlib import Path
from os import path
import numpy as np
from pprint import pprint
import pickle
import copy
from math import ceil, floor

%matplotlib inline
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-cb1285c4bd1a>:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
pd.set_option('mode.chained_assignment', None)

In [3]:
!pwd

/home/groups/russpold/uh2_analysis/Self_Regulation_Ontology_fMRI_2021/fmri_analysis/scripts/aim1_2ndlevel_regressors


# Constants

In [4]:
Max_SSD = 1
Min_SSD = 0
MAX_RT_STOP_TASK = 2.25

# Paths / Files

In [5]:
OAK = '/oak/stanford/groups/russpold' # CHANGE
#OAK = '/Volumes/russpold/' # CHANGE
BIDS_dir = path.join(OAK, 'data','uh2','aim1', 'BIDS')

In [6]:
# tasks to be looked at
tasks = ['ANT', 'discountFix', 'DPX', 'motorSelectiveStop', 'stopSignal', \
         'twoByTwo', 'stroop', 'WATT', 'CCTHot', 'surveyMedley']

In [7]:
# dropped subjects - if you find subjects with crappy beh data, put them in here as 'poor_performance_subjective_rating'
dropped_subjects = {
    's638': {'DPX': ['poor_performance_subjective_rating']},
    's586': {'DPX': ['poor_performance_subjective_rating'],
            'motorSelectiveStop': ['poor_performance_subjective_rating']},
    's650': {'ANT': ['poor_performance_subjective_rating'],
             'DPX': ['poor_performance_subjective_rating'],
             'stroop': ['poor_performance_subjective_rating']},
    's623': {'DPX': ['poor_performance_subjective_rating'],
             'twoByTwo': ['poor_performance_subjective_rating'],
             'surveyMedley' : ['poor_performance_subjective_rating']},
    's608': {'stroop': ['poor_performance_subjective_rating']},
    's172': {'WATT': ['poor_performance_subjective_rating']}
}

In [8]:
file_dict = {} #dict for storing task event file data 
for task in tasks: 
    file_dict[task] = glob(path.join(BIDS_dir, 'sub-s*', 'ses-*', 'func', f'*{task}*_events.tsv'))

# Helper Functions

In [9]:
def rt_summary(data_df, conditions, condition_column='trial_type'): 
    """
    calculates mean data for reaction time by condition
    
    Input: 
    - data_df::pandas df of the beh data across all subjects (1 large df of all subjects beh data)
    - conditions::list of conditions of interest for each task
    - condition_column::the column of the df we are interested in focusing on in terms of task conditions
    
    Output:
    - rt_df::pandas df of rt measures
    """
    
    # creates an empty df to input all the rt data for task conditions in question accross all subjects
    rt_df = pd.DataFrame()
    index = data_df['worker_id'].unique()
    
    columns = []
    if 'attention_network_task' in data_df['experiment_exp_id'].unique():
        for cond in conditions:
            columns.append(f'spatial_{cond}_rt')
            columns.append(f'double_{cond}_rt')
    else:    
        for cond in conditions:
            columns.append(f'{cond}_rt')
            
    rt_df = pd.DataFrame(index = index, columns = columns)
                                        
    # filters through every participant
    for subj in data_df['worker_id'].unique():
        
        # obtains avg rt for each subject across conditions
        for cond in conditions: 
    
            if 'ward_and_allport' in data_df['experiment_exp_id'].unique():
                subj_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj) & (data_df.loc[:,'planning'] == 1)]
            else:
                subj_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj)]
            
            if 'attention_network_task' in data_df['experiment_exp_id'].unique():
                double_mean_rt = subj_df.loc[(subj_df.loc[:, condition_column] == cond) & (subj_df.loc[:, 'cue'] == 'double')]['response_time'].mean()
                spatial_mean_rt = subj_df.loc[(subj_df.loc[:, condition_column] == cond) & (subj_df.loc[:, 'cue'] == 'spatial')]['response_time'].mean()
                rt_df.loc[(subj, f'double_{cond}_rt')] = double_mean_rt
                rt_df.loc[(subj, f'spatial_{cond}_rt')] = spatial_mean_rt
            else:
                mean_rt = subj_df.loc[subj_df.loc[:, condition_column] == cond]['response_time'].mean()
                rt_df.loc[(subj, f'{cond}_rt')] = mean_rt
            
            # obtains avg SSD for stop tasks
            if 'attention_netwstop_signalork_task' in data_df['experiment_exp_id'].unique() or 'motor_selective_stop_signal' in data_df['experiment_exp_id'].unique():
                rt_df.loc[(subj, 'mean_SSD')] = subj_df.loc[:, 'SS_delay'].mean()
            
        
    return rt_df


In [10]:
def acc_summary(data_df, conditions, condition_column='trial_type'): 
    """
    calculates summary data for task accuracy by condition
    obtains stop success and min/max ssd for stop signal and motor stop tasks
    obtains ommission rate for each task by condition
    
    Input: 
    - data_df::pandas df of the beh data across all subjects (1 large df of all subjects beh data)
    - conditions::list of conditions of interest for each task
    - condition_column::the column of the df we are interested in focusing on in terms of task conditions
    
    Output:
    - acc_df::pandas df of accuracy measures
    """
    
    # preprocessing: creates an empty df to input all the acc data for task conditions in question accross all subjects
    acc_df = pd.DataFrame()
    index = data_df['worker_id'].unique()
    
    columns = []
    if data_df['experiment_exp_id'].unique() == 'attention_network_task':
        for cond in conditions:
            columns.append(f'spatial_{cond}_acc')
            columns.append(f'double_{cond}_acc')
    else:    
        for cond in conditions:
            columns.append(f'{cond}_acc')
        
        columns.append('omission_rate')
        columns.append('overall_omission_rate')
        columns.append('truncation')
    
    acc_df = pd.DataFrame(index = index, columns = columns)
                                    
    # filters through every individual   
    for subj in data_df['worker_id'].unique():
        for cond in conditions:
            
            # obtains accuracy for conditions, ANT task gets treated differently because we seperate by cue as well
            if data_df['experiment_exp_id'].unique() == 'attention_network_task':
                subj_df_double = data_df.loc[(data_df.loc[:,'worker_id'] == subj) & (data_df[condition_column]==cond) & (data_df['cue']=='double')]
                subj_df_spatial = data_df.loc[(data_df.loc[:,'worker_id'] == subj) & (data_df[condition_column]==cond) & (data_df['cue']=='spatial')]
                double_acc = sum(subj_df_double['correct']) / len(subj_df_double['correct']) 
                spatial_acc = sum(subj_df_spatial['correct']) / len(subj_df_spatial['correct']) 
                acc_df.loc[(subj, f'spatial_{cond}_acc')] = spatial_acc
                acc_df.loc[(subj, f'double_{cond}_acc')] = double_acc
            else:
                if data_df['experiment_exp_id'].unique() == 'dot_pattern_expectancy':
                    subj_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj) & (data_df[condition_column]==cond) & (data_df['trial_id']=='probe')]
                else:
                    subj_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj) & (data_df[condition_column]==cond)]
                if cond in subj_df[condition_column].unique():
                    acc = sum(subj_df['correct']) / len(subj_df['correct']) 
                    acc_df.loc[(subj, f'{cond}_acc')] = acc
                else:
                    acc_df.loc[(subj, f'{cond}_acc')] = 0
            
            # obtains min/max ssd and stop success rate for stop tasks
            if data_df['experiment_exp_id'].unique() == 'stop_signal' or  data_df['experiment_exp_id'].unique() == 'motor_selective_stop_signal':
                beh_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj)]
                
                if data_df['experiment_exp_id'].unique() == 'stop_signal':
                    stop_trials = beh_df.loc[beh_df['trial_type'].isin(['stop_failure', 'stop_success'])]
                    acc_df.loc[(subj, 'max_SSD_count')] = (stop_trials.SS_delay.values == Max_SSD).sum()
                    acc_df.loc[(subj, 'min_SSD_count')] = (stop_trials.SS_delay.values == Min_SSD).sum()
                    acc_df.loc[(subj, 'mean_SSD')] = (stop_trials.SS_delay.values).mean()
                    acc_df.loc[(subj, 'SSRT')] = calc_SSRT(beh_df, task = 'stop_signal')
                    
                elif data_df['experiment_exp_id'].unique() == 'motor_selective_stop_signal':
                    stop_trials = beh_df.loc[beh_df['trial_type'].isin(['crit_stop_success', 'crit_stop_failure'])]
                    acc_df.loc[(subj, 'max_SSD_count')] = (stop_trials.SS_delay.values == Max_SSD).sum()
                    acc_df.loc[(subj, 'min_SSD_count')] = (stop_trials.SS_delay.values == Min_SSD).sum()
                    acc_df.loc[(subj, 'mean_SSD')] = (stop_trials.SS_delay.values).mean()
                    acc_df.loc[(subj, 'SSRT')] = calc_SSRT(beh_df, task = 'motor_selective_stop_signal')
                    
                    crit_go_trials = beh_df[(beh_df.trial_type == 'crit_go')]
                    noncrit_nosignal_trials = beh_df[(beh_df.trial_type == 'noncrit_nosignal')]
                    noncrit_signal_trials = beh_df[(beh_df.trial_type == 'noncrit_signal')]
                    
                    acc_df.loc[(subj, 'noncrit_signal_omission')] = (noncrit_signal_trials.key_press.values == -1).sum() / len(noncrit_signal_trials)
                    acc_df.loc[(subj, 'noncrit_nosignal_omission')] = (noncrit_nosignal_trials.key_press.values == -1).sum() / len(noncrit_nosignal_trials)
                    acc_df.loc[(subj, 'crit_go_omission')] = (crit_go_trials.key_press.values == -1).sum() / len(crit_go_trials)
                    
                if set(data_df[data_df['worker_id'] == subj].key_press) == {-1.0, 71.0, 89.0}:
                    if data_df['experiment_exp_id'].unique() == 'stop_signal':
                        stop_success = len(beh_df[beh_df.trial_type == 'stop_success'])
                        stop_failure = len(beh_df[beh_df.trial_type == 'stop_failure'])
                        
                    elif data_df['experiment_exp_id'].unique() == 'motor_selective_stop_signal':
                        stop_success = len(beh_df[beh_df.trial_type == 'crit_stop_success'])
                        stop_failure = len(beh_df[beh_df.trial_type == 'crit_stop_failure'])

                    total = stop_success + stop_failure
                    stop_success_rate = stop_success / total
                    acc_df.loc[(subj, 'stop_success_rate')] = stop_success_rate
                else:
                    acc_df.loc[(subj, 'stop_success_rate')] = 0
        
        # obtains omission rate for current data
        subj_df = data_df.loc[(data_df.loc[:,'worker_id'] == subj)]
        
        acc_df.loc[(subj, 'omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        
        # obtains omission rate from any previous iteration in the event the data was truncated
        if 'overall_omission_rate' in subj_df.columns:
            acc_df.loc[(subj, 'overall_omission_rate')] = subj_df['overall_omission_rate'].unique()[0]
        else:
            acc_df.loc[(subj, 'overall_omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        
        # obtains truncation rate if data was truncated
        if 'truncation' in subj_df.columns:
            acc_df.loc[(subj, 'truncation')] = subj_df['truncation'].unique()[0]
        else:
            acc_df['truncation'] = np.nan
            
    return acc_df

In [11]:
def calc_SSRT(df, task = None, max_rt = MAX_RT_STOP_TASK):
    if task == 'stop_signal':
        go_trials = df.query('trial_type == "go"')
        stop_trials = df.query('trial_type == "stop_success" or trial_type == "stop_failure"')
        go_replacement_df = go_trials.where(go_trials['response_time'] != -1, max_rt)
        sorted_go = go_replacement_df.response_time.sort_values(ascending = True)
        prob_stop_failure = (1-stop_trials.stopped.mean())
        nth = prob_stop_failure*(len(sorted_go)-1)
        index = [floor(nth), ceil(nth)]
        SSRT = sorted_go.iloc[index].mean() - stop_trials.SS_delay.mean()
        return SSRT
    
    elif task == 'motor_selective_stop_signal':
        
        go_trials = df.query('trial_type == "crit_go"')
        stop_trials = df.query('trial_type == "crit_stop_failure" or trial_type == "crit_stop_success"')
        go_replacement_df = go_trials.where(go_trials['response_time'] != -1, max_rt)
        sorted_go = go_replacement_df.response_time.sort_values(ascending = True)
        prob_stop_failure = (1-stop_trials.stopped.mean())
        nth = prob_stop_failure*(len(sorted_go)-1)
        index = [floor(nth), ceil(nth)]
        SSRT = sorted_go.iloc[index].mean() - stop_trials.SS_delay.mean()
        return SSRT      
    

In [12]:
def omission_rate(df, task):
    """
    function to calc the rate of ommisions within the data
    
    Input: 
    - df::pandas df of the beh data for an individual subject
    - task::str::task that the data belongs to
    
    Output:
    - non_response_rate::int::scalar value representing non-response rate of subject for task
    """
    
    # checks that the data is not empty or missing
    if df.empty == False: 
        
        # obtains omission or non-response rate for specific task
        if (task == 'stop_signal') or (task == 'stopSignal'):
            go_trials = df[(df.trial_type == 'go')]
            non_response_rate = len(go_trials[go_trials.key_press == -1]) / len(df[df.trial_type == 'go'])
        elif (task == 'motor_selective_stop_signal') or (task == 'motorSelectiveStop'):
            crit_go_trials = df[(df.trial_type == 'crit_go')]
            noncrit_nosignal_trials = df[(df.trial_type == 'noncrit_nosignal')]
            noncrit_signal_trials = df[(df.trial_type == 'noncrit_signal')]
            go_trials = pd.concat([crit_go_trials, noncrit_nosignal_trials, noncrit_signal_trials])
            non_response_rate = len(go_trials[go_trials.key_press == -1]) / len(df[(df.trial_type != 'crit_stop_success') & (df.trial_type != 'crit_stop_failure')])
        elif (task == 'survey_medley') or (task == 'surveyMedley'):
            non_response_rate = len(df[df.coded_response == 0]) / len(df.coded_response)
        elif (task == 'columbia_card_task_fmri') or (task == 'CCTHot'):
            non_response_rate = len(df[(df['action'] != -1) & (df['key_press'] == -1)]) / len(df[df['action'] != -1])
        elif (task == 'ward_and_allport') or (task == 'WATT'):
            non_response_rate = len(df.loc[(df['trial_id'] != 'feedback') & (df['key_press'] == -1)]) / len(df[df['trial_id'] != 'feedback'])            
        else:
            non_response_rate = len(df[df.key_press == -1]) / len(df.key_press)
            
    else:
        non_response_rate = 1
        
    return non_response_rate

In [13]:
def CCTHot_EV_acc(task_df):
    """
    Function to calc accuracy for CCTHot task
    We compute accuracy as the proportion of rewarded trials.
    We also compute ommision rate for the task data
    
    Input: 
    - task_df::pandas dataframe:: beh data for task accross all subjects
    
    Output:
    - acc_df::pandas df of accuracy measures for CCTHot
    """
    
    # preprocessing
    task_df['EV_new'] = task_df['gain_amount']*task_df['gain_probability'] + task_df['loss_amount']*task_df['loss_probability']
    task_df['correct'] = np.nan
    # incorrect to draw card with negative EV, end round with positive EV
    task_df.loc[(task_df.action=='draw_card') & (task_df.EV_new < 0), 'correct'] = 0
    task_df.loc[(task_df.action=='end_round') & (task_df.EV_new >= 0), 'correct'] = 0
    # correct to draw card with positive EV, end round with negative EV
    task_df.loc[(task_df.action=='draw_card') & (task_df.EV_new >= 0), 'correct'] = 1
    task_df.loc[(task_df.action=='end_round') & (task_df.EV_new < 0), 'correct'] = 1
    
    # creates df to store acc data
    acc_df = pd.DataFrame()
    index = task_df['worker_id'].unique()
    acc_df = pd.DataFrame(index = index, columns = ['acc'])
    
    # iterate through every subject to obtain individual acc/ommission rate
    for subj in task_df.worker_id.unique():
        acc_df.loc[(subj, 'acc')] = task_df.loc[task_df.worker_id == subj, 'correct'].mean()
        
        subj_df = task_df.loc[(task_df.loc[:,'worker_id'] == subj)]
        
        # overall omission rate doesn't need to be included, so I may remove and see if the code works
        acc_df.loc[(subj, 'CCTHot_overall_omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        acc_df.loc[(subj, 'omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        
        # truncation is not being done on CCTHot due to the difference in accuracy measures we obtain
        # included due to errors flaggin when other functions iterate through the final output - may take out
        acc_df.loc[(subj, 'CCTHot_truncation')] = np.nan
        
    return acc_df

In [14]:
def WATT_acc(task_df):
    """
    Function to calc accuracy for WATT task
    We compute accuracy as extra moves taken.
    We also compute ommision rate for the task data
    
    Input: 
    - task_df::pandas dataframe:: beh data for task accross all subjects
    
    Output:
    - acc_df::pandas df of accuracy measures for WATT
    """
    
    # creates df to store acc data
    acc_df = pd.DataFrame()
    index = task_df['worker_id'].unique()
    acc_df = pd.DataFrame(index = index, columns = ['acc'])
    
    # iterated through every subject to obtain individual acc/ommission rate
    for subj in task_df.worker_id.unique():
    
        subj_df = task_df.loc[task_df.worker_id == subj,].copy()
        
        if subj_df.empty:
            print(f"subj_df is empty for subject {subj}. Skipping...")
            continue
    
        
        subj_df = subj_df.reset_index(drop=True)
        p_idx = subj_df[subj_df.planning==1].index
        f_idx = subj_df[subj_df.trial_id=='feedback'].index
        assert len(p_idx)==len(f_idx)
        moves = f_idx - p_idx
        moves = moves/2
        unnecessary_moves = moves - 3
                
        # overall omission rate doesn't need to be included, so I may remove and see if the code works
        unique_values = subj_df['experiment_exp_id'].unique()
        if len(unique_values) == 0:
            print(f"No unique values for 'experiment_exp_id' for subject {subj}. Skipping...")
            continue
        acc_df.loc[(subj, 'WATT_overall_omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        acc_df.loc[(subj, 'omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        
        acc_df.loc[(subj, 'acc')] = np.mean(unnecessary_moves)
        
        # truncation is not being done on WATT due to the difference in accuracy measures we obtain
        # included due to errors flaggin when other functions iterate through the final output - may take out
        acc_df.loc[(subj, 'WATT_truncation')] = np.nan
        
    return acc_df

In [15]:
def discount_acc(task_df):
    """
    Function to calc accuracy for discountFix task
    We compute accuracy as the proportion of larger later trials.
    We also compute ommision rate for the task data
    
    Input: 
    - task_df::pandas dataframe:: beh data for task accross all subjects
    
    Output:
    - acc_df::pandas df of accuracy measures for discountFix
    """
    
    # preprocessing
    task_df['correct'] = np.nan
            
    task_df.loc[task_df.choice=='larger_later', 'correct'] = 1
    task_df.loc[task_df.choice=='smaller_sooner', 'correct'] = 0
    
    # creates df to store acc data
    acc_df = pd.DataFrame()
    index = task_df['worker_id'].unique()
    acc_df = pd.DataFrame(index = index, columns = ['acc'])
    
    # iterated through every subject to obtain individual acc/ommission rate
    for subj in task_df.worker_id.unique():
        acc_df.loc[(subj, 'acc')] = task_df.loc[task_df.worker_id == subj, 'correct'].mean()
        
        subj_df = task_df.loc[(task_df.loc[:,'worker_id'] == subj)]
        
        # overall omission rate doesn't need to be included, so I may remove and see if the code works
        acc_df.loc[(subj, 'omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        acc_df.loc[(subj, 'discountFix_overall_omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        
        # truncation is not being done on discount due to the difference in accuracy measures we obtain
        # included due to errors flaggin when other functions iterate through the final output - may take out
        if 'discountFix_truncation' not in task_df.columns:
            acc_df.loc[(subj, 'discountFix_truncation')] = np.nan
        else:
            acc_df.loc[(subj, 'discountFix_truncation')] == subj_df['truncation'].unique()[0]
        
        
    return acc_df 

In [16]:
def surveyMedley_acc(task_df):
    """
    Function to calc accuracy for surveyMedley task
    We compute freuency of responses across all options.
    We also compute ommision rate for the task data
    
    Input: 
    - task_df::pandas dataframe:: beh data for task accross all subjects
    
    Output:
    - acc_df::pandas df of accuracy measures for CCTHot
    """
    
    # creates df to store acc data
    acc_df = pd.DataFrame()
    index = task_df['worker_id'].unique()
    acc_df = pd.DataFrame(index = index, columns = ['omission_rate', 'surveyMedley_omission_rate_overall'])
    
    # iterated through every subject to obtain individual acc/ommission rate
    for subj in task_df.worker_id.unique():
        subj_df = task_df.loc[(task_df.loc[:,'worker_id'] == subj)]
        
        if subj_df.empty == False: 
            total = len(subj_df.coded_response)
            
            one = len(subj_df[subj_df.coded_response == 1]) / total
            two = len(subj_df[subj_df.coded_response == 2]) / total
            three = len(subj_df[subj_df.coded_response == 3]) / total
            four = len(subj_df[subj_df.coded_response == 4]) / total
            five = len(subj_df[subj_df.coded_response == 5]) / total
            
        else:
            one = 0
            two = 0
            three = 0 
            four = 0
            five = 0
        
        
        # overall omission rate doesn't need to be included, so I may remove and see if the code works
        acc_df.loc[(subj, 'omission_rate')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
        acc_df.loc[(subj, 'surveyMedley_omission_rate_overall')] = omission_rate(subj_df, subj_df['experiment_exp_id'].unique()[0])
         
        acc_df.loc[(subj, '1')] = one
        acc_df.loc[(subj, '2')] = two
        acc_df.loc[(subj, '3')] = three
        acc_df.loc[(subj, '4')] = four
        acc_df.loc[(subj, '5')] = five
        
        # truncation is not being done on surveyMedley due to the difference in accuracy measures we obtain
        # included due to errors flaggin when other functions iterate through the final output - may take out
        acc_df.loc[(subj, 'surveyMedley_truncation')] = np.nan
        
    return acc_df

In [17]:
def outlier_highlight(data_df, outlier_threshold = 3):
    """
    Function to highlight a data frame containing accuracy for task conditions of all subjects
    Iterates through df cell by cell and highlightes accordingly
    Highlighting:
            blue: missing data
            orange: more than half of the tasks are missing
            yellow: outlier
            green: recommended to drop, see flag for further details
    
    Input: 
    - data_df::pandas dataframe:: acc of task conditions for all subjects
    - threshold::int::scalar value representing outlier threshold (standard deviations away from mean)
    Output:
    - style::pandas dataframe:: a df that contrains highlighting options to use with pandas styler
    """
    
    # creating empty df to populate with highlight options
    columns = list(data_df)
    style = data_df.copy()
    
    # iterates through all columns in df
    for i in columns:
        label = i.split("_")
        task = label[0]
        
        if ('overall_mean' in data_df.index) or ('overall_std' in data_df.index):
            mean = np.mean(data_df[i][2:])
            std = np.std(data_df[i][2:])
        else:
            mean = np.mean(data_df[i])
            std = np.std(data_df[i])
        
        # iterates through each individual cell within colums and checks if its an outlier or in drop dictionary
        # if subject is in the dropped dictionary for that task it colors accordingly
        for index, row in data_df[[i]].iterrows():
            if index not in ['overall_std', 'overall_mean']:
                if index in dropped_subjects.keys():
                    if task in dropped_subjects[index].keys():
                        if 'missing_beh_data' in dropped_subjects[index][task]:
                            style.loc[index, i] = "background-color: blue"
                        elif 'missing_more_than_half_the_tasks' in dropped_subjects[index][task]:
                            style.loc[index, i] = "background-color: orange"
                        else:
                            style.loc[index, i] = "background-color: green"
                    elif (abs((row[0]-mean)/std) > outlier_threshold) & (i not in [f'{task}_truncation',f'{task}_overall_omission_rate']):
                        style.loc[index, i] = "background-color: yellow"
                    else:
                        style.loc[index, i] = ""
                elif (abs((row[0]-mean)/std) > outlier_threshold) & (i not in [f'{task}_truncation',f'{task}_overall_omission_rate']):
                    style.loc[index, i] = "background-color: yellow"
                else:
                    style.loc[index, i] = ""
            else:
                style.loc[index, i] = ""
        
    return style

In [18]:
def outlier(data_df, outlier_threshold = 3):
    """
    Function to obtain outliers in the acc data
    Iterates through df cell by cell and obtains outliers
    
    Input: 
    - data_df::pandas dataframe:: acc of task conditions for all subjects
    - threshold::int::scalar value representing outlier threshold (standard deviations away from mean)
    Output:
    - outliers::dictionary:: a dictionary that contains the subject and corresponding columns if they are outliers (pass the chosen threshold)
    """
    
    columns = list(data_df)
    outliers = {}
    for column in columns:
        label = column.split("_")
        task = label[0]
        
        # obtains mean and st.d for column across all subjects for flagging
        mean = np.mean(data_df[column])
        std = np.std(data_df[column])
        
        # iterates throuch each column comparing cells z score to threshold
        # appends outliers to dict / doesnt check overall std, truncation, or overall omission
        for index, row in data_df[[column]].iterrows():
            if (abs((row[0]-mean)/std) > outlier_threshold)  & (index != 'overall_std') & (column not in [f'{task}_truncation', f'{task}_overall_omission_rate']):
                if index in outliers.keys():
                    outliers[index].append(column)
                else:
                    outliers[index] = [column]
        
    return outliers

In [19]:
def truncation_check(df, task, trial_threshold = 3, omission_threshold = .50):
    """
    Function to truncate data 
    - if there are more than 3 ommisions in a row then all the data after those three omissions will be put into another df.
    - if the omission rate of that data is > .5 we truncate the data
    - otherwise, the data is left alone
    
    Input: 
    - df::pandas dataframe:: beh data for a single subject
    - task::str::name of task the data belongs to
    - trial_threshold::int::scaler of the number of omissions in a row that will signal truncation
    - omission_threshold::float::precentage of omission that we look for to flag truncation
    Output:
    - df::pandas dataframe:: untouched data, without any trials dropped - was not truncated
    - truncated_df::pandas dataframe:: data after truncation
    - truncation_rate::float:: percentage of trials that were dropped
    """
    
    columns = list(df)
    
    # list of responses associated with each task
    if task == 'stopSignal':
        go_trials = df[df.trial_type == 'go']
        resp = go_trials['key_press']
    elif task == 'motorSelectiveStop':
        go_trials = df[(df.trial_type != 'crit_stop_success') & (df.trial_type != 'crit_stop_failure')]
        resp = go_trials['key_press']
    else:
        resp = df['key_press']
    
    # iterated through every response checking for omissions
    count = 0
    for i, row in enumerate(resp):
        
        # ommisions are either -1 or nan
        if row == -1:
            count += 1
        else:
            count = 0
        
        # if we find a series of omissions (n=trial_threshold) we inspect the data
        if count == trial_threshold:
            index = (i+1) - trial_threshold
            
            # obtains all the data starting from the omissions and obtains a ref df then obtains their omission rate
            if task == 'stopSignal':
                ref_df = go_trials[index:]
                omission_rate = len(ref_df[ref_df.key_press == -1]) / len(ref_df[ref_df.trial_type == 'go'])
                
            elif task == 'motorSelectiveStop':
                ref_df = df[index:]
                omission_rate = len(ref_df[ref_df.key_press == -1]) / len(ref_df)
                
            else:
                ref_df = df[index:]
                omission_rate = len(ref_df[ref_df.key_press == -1]) / len(ref_df.key_press)
                   
            # if the ommision rate of the ref df is more than half we truncate
            if omission_rate > omission_threshold:
                truncated_df = df[:index]
                truncation_rate = len(ref_df)/len(df)
                
                return truncated_df, truncation_rate
            
            elif len(ref_df)/len(resp) == 1:
                    lst = list(df.key_press)
                    cnt = 0
                    while lst[cnt] == -1:
                        cnt += 1
                    
                    truncated_df = df[cnt:]
                    truncation_rate = 1 - (len(truncated_df)/len(df))
                    return truncated_df, truncation_rate
            
            print(f'failed to truncate omission_rate was less than half of the df: {omission_rate}')
            truncation_rate = 0
            return df, truncation_rate
        
    print(f'failed to truncate no sequence of 3 ommisions found.')
    truncation_rate = 0
    return df, truncation_rate
    

In [20]:
def outlier_analysis(avg_info, data, outlier_threshold = 3, trial_threshold = 3, omission_threshold = .50):
    """
    Function to analyze the outliers that were previously flagged
    We take all the flagged outliers and check their omission for possible truncation
    
    Input: 
    - avg_info::pandas dataframe:: contains the acc measures for each task condition of interest for all subjects
    - data::dict:: dictionary of df's containing all the beh data for all tasks 
    - threshold::int::threshold to be passed through to the outlier function
    Output:
    - data::dict:: dictionary of updated beh data for all tasks (updated with truncation)
    - nontruncated_outlier::dict:: contains subjects that have not been truncated because they are missing too many trials
    """
    
    # obtains outliers
    outliers = outlier(avg_info, outlier_threshold)
    nontruncated_outlier = {}
    
    # obtains all the unique tasks we have had outliers for
    for subject in outliers.keys():
        tasks = []
        for flag in outliers[subject]:
            label = flag.split("_")
            task = label[0]
            condition = '_'.join(label[1:-1])
            tasks.append(task)

        unique_tasks = list(set(tasks))
        
        # iterates through each outlier
        for task in unique_tasks:
            
            # obtains subject data
            subj_df = data[task].loc[(data[task].loc[:,'worker_id'] == subject)]
            
            # adds truncation and overall omission columns to populate if needed to
            if 'truncation' not in data[task].columns:
                data[task]['truncation'] = np.nan
            
            if 'overall_omission_rate' not in data[task].columns:
                data[task]['overall_omission_rate'] = np.nan
            
            # obtains omission rate 
            non_response_rate = omission_rate(subj_df, task)
            
            # checks if omissions are to extensive for truncation wihtin outliers
            if (non_response_rate < .50) & (task not in ['WATT', 'surveyMedley']):
                print(f'subject: {subject} had nonresponses truncated for {task}: {non_response_rate}')
                
                # checks to see if data can be truncated
                response_df, truncation_rate = truncation_check(subj_df, task, trial_threshold, omission_threshold)
                
                if truncation_rate == 1:
                    if subject not in nontruncated_outlier.keys():
                        nontruncated_outlier[subject] = [task]
                    else:
                        nontruncated_outlier[subject].append(task)
                
                # obtains omission rate before truncation - overall omission
                response_df['overall_omission_rate'] = omission_rate(subj_df, task)
                
                response_df['truncation'] = truncation_rate
                data[task] = data[task][data[task]['worker_id'] != subject]
                data[task] = data[task].append(response_df)
                
            
            # of omission rate is initially over half, they are dropped - not enough data
            elif (non_response_rate > .50):
                print(f'subject: {subject} had a nonresponse rate of {non_response_rate} for {task}')
                
                subj_df['overall_omission_rate'] = omission_rate(subj_df, task)
                
                
                subj_df['truncation'] = 0
                data[task] = data[task][data[task]['worker_id'] != subject]
                data[task] = data[task].append(subj_df)
                
                if subject not in nontruncated_outlier.keys():
                    nontruncated_outlier[subject] = [task]
                else:
                    nontruncated_outlier[subject].append(task)
                
    return data, nontruncated_outlier

In [21]:
def overall_analysis(data, non_truncated, dropped = dropped_subjects, tasks = tasks):
    """
    Function to analyze the final acc df after analyzing outliers. The final acc df contains all acc measures for all subjects .
    Obtains a dictionary of dropped subjects and flags for why they are recommended to be dropped 
    We iterate through different conditions, flagging for the following:
        
        'greater_than_50_percent_nonresponse_rate': the subject is missing more than half the data due to omissions
        'less_than_25_percent_stop_sucess_rate': The subjects stop success rate is greater than 75% (stop/motor)
        'more_than_75_percent_stop_sucess_rate': The subjects stop success rate is less than 25% (stop/motor)
        'missing_beh_data': subject is missing the beh data for that task
        'truncated_more_than_half_data': subject had data truncated, and the data that was truncated accounted for more than 50% of their data
        'missing_more_than_half_the_tasks': subject flagged more than 4 tasks for reasons listed above
    
    Input: 
    - data::pandas dataframe:: contains the acc measures for each task condition of interest for all subjects
    - dropped::dict:: dictionary of dictionares containing subjects - tasks - flags for why tasks are recommened to be dropped
    - tasks::list:: tasks we are looking at
    Output:
    - data::pandas dataframe:: contains each subject acc accross all task conditions of interest
    - dropped::dict:: dictionary of dictionares containing subjects - tasks - flags for why tasks are recommened to be dropped
    """
    
    columns = list(data)
    stop_success = ['motorSelectiveStop_stop_success_rate', 'stopSignal_stop_success_rate']
    motor_as_simple_stop_flag = ['motorSelectiveStop_noncrit_signal_omission']
    
    # iterated through the motor stop non crit signal flagging for acc < .4 [cutoff chosen by patrick]
    for column in motor_as_simple_stop_flag:
        label = column.split("_")
        task = label[0]
        
        for index, row in data[[column]].iterrows():
            if index != 'overall_std': 
                if (row[0] > .347):
                    print(f'{index} dropped for having a noncrit signal omission rate of {row[0]}')
                    if index not in dropped.keys():
                        if row[0] > .347:
                            dropped[index] = {task: ['greater_than_35_precent_om_noncrit_signal_motorstop']}
                    else:
                        if task not in dropped[index].keys():
                            if row[0] > .347:
                                dropped[index][task] = ['greater_than_35_precent_om_noncrit_signal_motorstop']
                        else:
                            if row[0] > .347:
                                dropped[index][task].append('greater_than_35_precent_om_noncrit_signal_motorstop')
        
    
    # iterates through the non_truncated dict to see who has an ommision rate > .5 before truncation
    for sub, i in non_truncated.items():
        for task in i:
            if sub not in dropped.keys():
                dropped[sub] = {task: ['greater_than_50_percent_nonresponse_rate']}
                                    
            else:
                if task not in dropped[sub].keys():
                    dropped[sub][task] = ['greater_than_50_percent_nonresponse_rate']
                else:
                    dropped[sub][task].append('greater_than_50_percent_nonresponse_rate')
    
    # iterates through data to flag subjects with stop success < .25 or > .75
    for column in stop_success:
        label = column.split("_")
        task = label[0]

        for index, row in data[[column]].iterrows():
            if index != 'overall_std': 
                if (row[0] < .25) or (row[0] > .75):
                    print(f'{index} dropped for having a stop success of {row[0]}')
                    if index not in dropped.keys():
                        if row[0] < .25:
                            dropped[index] = {task: ['less_than_25_percent_stop_sucess_rate']}
                        elif row[0] > .75:
                            dropped[index] = {task: ['more_than_75_percent_stop_sucess_rate']}
                    else:
                        if task not in dropped[index].keys():
                            if row[0] < .25:
                                dropped[index][task] = ['less_than_25_percent_stop_sucess_rate']
                            elif row[0] > .75:
                                dropped[index][task] = ['more_than_75_percent_stop_sucess_rate']
                        else:
                            if row[0] < .25:
                                dropped[index][task].append('less_than_25_percent_stop_sucess_rate')
                            elif row[0] > .75:
                                dropped[index][task].append('more_than_75_percent_stop_sucess_rate')
    
    # iterates through data to flag all tasks with missing data 
    for column in columns:

        label = column.split("_")
        task = label[0]

        for index, row in data[[column]].iterrows():
            if (np.isnan(row[0])) & (column not in [f'{task}_truncation', f'{task}_overall_omission_rate', f'{task}_omission_rate']):
                if index in dropped.keys():
                    if task not in dropped[index].keys():
                        dropped[index][task] = ['missing_beh_data']
                    else:
                        if 'missing_beh_data' not in dropped[index][task]:
                            dropped[index][task].append('missing_beh_data')
                else:
                    dropped[index] = {task: ['missing_beh_data']}
    
    # iterates through data to flag any subjects that had more than half of their data truncated
    for task in tasks:
        for index, row in data[[f'{task}_truncation']].iterrows():
            if row[0] > .5:
                if index in dropped.keys():
                    if task not in dropped[index].keys():
                        dropped[index][task] = ['truncated_more_than_half_data']
                    else:
                        if 'truncated_more_than_half_data' not in dropped[index][task]:
                            dropped[index][task].append('truncated_more_than_half_data')
                else:
                    dropped[index] = {task: ['truncated_more_than_half_data']}
            
    for subject in dropped.keys():
        flagged = len(dropped[subject])
        if flagged >= 5:
            for task in tasks:
                if task not in dropped[subject].keys():
                    dropped[subject][task] = ['missing_more_than_half_the_tasks']
                else:
                    if 'missing_more_than_half_the_tasks' not in dropped[subject][task]:
                        dropped[subject][task].append('missing_more_than_half_the_tasks')
                    

    return data, dropped

In [22]:
def format_data(data, rt_flag = False, acc_flag = False, overall_metrics = True):
    """
    Function to format the output df appropriatley. This function takes the acc or rt data obtains in the summary functions
    and then puts them all into one complete df containing all the subjects and all the desired condition columns.
    
    Input: 
    - data::dict:: contains the acc measures for each task condition of interest for all subjects
    - rt_flag::bool:: flag if you want rt measures for all tasks
    - acc_flag::bool:: flag if you want acc measures for all tasks
    - overall_metrics::bool:: flag if you want overall mean and st.d for task conditions(columns of data) - avg across all subjects
    Output:
    - df::pandas dataframe:: contains rt/acc measures (depending on flag) for all subjects and task conditions of interest
    - Final_df::pandas dataframe:: contains rt/acc measures (depending on flag) for all subjects including overall metrics
    """
    
    # obtains rt metrics
    if rt_flag == True:

        df = None
        # iterates through all tasks and replaces certain columns to be more descriptive
        for key in data.keys():
            task_df = pd.DataFrame.from_dict(data[key])
            if key == 'WATT':
                task_df = task_df.rename(columns = {'PA_with_intermediate_rt':'PA_with_rt'})
                task_df = task_df.rename(columns = {'PA_without_intermediate_rt':'PA_without_rt'})
            elif key == 'CCTHot':
                task_df = task_df.rename(columns = {'poldrack-single-stim_rt':'overall_rt'})

            task_df.columns = [f'{key}_'+col for col in task_df.columns]
            if df is None:
                df = task_df
            else:
                df = pd.concat([df, task_df], axis=1, sort=False)

        df = df.sort_index()
    
    # obtains acc metrics
    if acc_flag == True:
            
        df = None
        
        # iterates through all tasks and replaces certain columns to be more descriptive
        for key in data.keys():
            task_df = pd.DataFrame.from_dict(data[key])
            if key == 'WATT':
                task_df = task_df.rename(columns = {'acc':'WATT_extraMoves'})
                task_df = task_df.rename(columns = {'omission_rate':'WATT_omission_rate'})
            elif key == 'CCTHot':
                task_df = task_df.rename(columns = {'acc':'CCTHot_proportionRewardedTrials'})
                task_df = task_df.rename(columns = {'omission_rate':'CCTHot_omission_rate'})
            elif key == 'discountFix':
                task_df = task_df.rename(columns = {'acc':'discountFix_largerlaterpercentage'})
                task_df = task_df.rename(columns = {'omission_rate':'discountFix_omission_rate'})
            elif key == 'surveyMedley':
                task_df = task_df.rename(columns = {'omission_rate':'surveyMedley_omission_rate'})
                task_df = task_df.rename(columns = {'1':'surveyMedley_1_option_rate'})
                task_df = task_df.rename(columns = {'2':'surveyMedley_2_option_rate'})
                task_df = task_df.rename(columns = {'3':'surveyMedley_3_option_rate'})
                task_df = task_df.rename(columns = {'4':'surveyMedley_4_option_rate'})
                task_df = task_df.rename(columns = {'5':'surveyMedley_5_option_rate'})
            else:
                task_df.columns = [f'{key}_'+col for col in task_df.columns]
            if df is None:
                df = task_df
            else:
                df = pd.concat([df, task_df], axis=1, sort=False)
        
        df = df.sort_index()

        # flip WATT, essentially from errors to acc
        if 'WATT_extraMoves' in df.columns:
            df['WATT_extraMoves'] = df['WATT_extraMoves'].max() - df['WATT_extraMoves']
    
    # obtains overall avg/std for every column
    if overall_metrics == True:
        
        # getting average rt accross all subjects for each condition
        column_avg = df.mean(axis = 0).tolist()
        avg = pd.DataFrame([column_avg], columns=df.columns.tolist())
        avg.index = ['overall_mean']

        # getting std across all subjects for each condition
        column_std = df.std(axis = 0).tolist()
        std = pd.DataFrame([column_std], columns=df.columns.tolist())
        std.index = ['overall_std']

        overall_metrics = pd.concat([avg, std])
        Final_df = pd.concat([overall_metrics, df])
        return Final_df
    
    else:
        return df

# Load all task data into dict

In [23]:
alldata_dict = {}
for task in tasks: 
    task_df = pd.DataFrame()
    for file in file_dict[task]: 
        df = pd.DataFrame()
        df = pd.read_csv(file, sep='\t')
        df.dropna(subset=['worker_id'], inplace=True)
        task_df = task_df.append(df, sort=True)   
    
    #create ANT conditions
    if task == 'ANT':
        task_df['trial_type'] = task_df['cue'] + '_' + task_df['flanker_type']
        
    if task in ['motorSelectiveStop', 'stopSignal',]:
        task_df['correct'] = task_df.apply(lambda row: 1 if row['key_press'] == row['correct_response'] else 0, axis=1)

        
    alldata_dict[task] = task_df

# RT Summary stats for each condition of each task

In [24]:
condition_dict = {
    'twoByTwo': alldata_dict['twoByTwo']['switch_type'].unique().tolist(),
    'stopSignal': ['go', 'stop_failure'],
    'motorSelectiveStop': ['noncrit_signal', 'noncrit_nosignal', 'crit_stop_failure', 'crit_go'],
    'ANT': alldata_dict['ANT']['flanker_type'].unique().tolist(),
    'discountFix': ['smaller_sooner', 'larger_later'],
    'stroop': alldata_dict['stroop']['trial_type'].unique().tolist(),
    'WATT': alldata_dict['WATT']['condition'].unique().tolist(),
}
column_dict = {
    'twoByTwo': 'switch_type',
    'ANT': 'flanker_type', 
    'discountFix': 'choice',
    'WATT': 'condition',
}

In [25]:
rt_dict = {}
for task in tasks:         
    if task != 'surveyMedley':
        if task in condition_dict:
            conditions = condition_dict[task]
            cond_col = column_dict.get(task, 'trial_type')
        else:
            try:
                conditions = alldata_dict[task]['trial_type'].unique().tolist()
                cond_col = 'trial_type'
            except KeyError:
                print(f"Skipping {task} due to missing 'trial_type' column.")
                continue
                
        rt_dict[task] = rt_summary(alldata_dict[task], conditions, cond_col)
    
FINAL_task_rt_df = format_data(rt_dict, rt_flag = True, acc_flag = False, overall_metrics = True)

Skipping CCTHot due to missing 'trial_type' column.


In [26]:
FINAL_task_rt_df

,ANT_spatial_incongruent_rt,ANT_double_incongruent_rt,ANT_spatial_congruent_rt,ANT_double_congruent_rt,discountFix_smaller_sooner_rt,discountFix_larger_later_rt,DPX_AX_rt,DPX_AY_rt,DPX_BX_rt,DPX_BY_rt,motorSelectiveStop_noncrit_signal_rt,motorSelectiveStop_noncrit_nosignal_rt,motorSelectiveStop_crit_stop_failure_rt,motorSelectiveStop_crit_go_rt,motorSelectiveStop_mean_SSD,stopSignal_go_rt,stopSignal_stop_failure_rt,twoByTwo_task_switch_rt,twoByTwo_cue_stay_rt,twoByTwo_cue_switch_rt,stroop_congruent_rt,stroop_incongruent_rt,WATT_nan_rt,WATT_UA_with_intermediate_rt,WATT_UA_without_intermediate_rt,WATT_PA_with_rt,WATT_PA_without_rt
overall_mean,0.588174,0.637917,0.576917,0.604836,1.52648,1.519464,0.53189,0.639819,0.452191,0.465915,0.628662,0.598228,0.644335,0.712547,0.350570,0.666337,0.590753,0.77732,0.711777,0.756088,0.642193,0.740818,NaN,4.65841,3.557401,3.774927,3.289199
overall_std,0.272099,0.111176,0.103228,0.111112,0.395192,0.422202,0.114112,0.116572,0.138351,0.141108,0.13123,0.126332,0.131609,0.170806,0.185483,0.162752,0.115952,0.133666,0.115116,0.13684,0.106519,0.14089,NaN,1.786598,1.599828,1.635261,1.669714
s061,NaN,NaN,NaN,NaN,1.860722,1.70206,0.595705,0.73125,0.534125,0.500792,0.602735,0.575933,0.6106,0.606605,0.354000,0.734613,0.739037,0.859561,0.803879,0.893098,0.650292,0.851125,NaN,4.7735,2.7765,2.937583,2.506292
s130,0.500563,0.579719,0.455771,0.569724,1.621867,1.580556,0.96157,0.985167,0.8805,0.862375,0.540102,0.527079,0.579,0.714,0.445600,0.699173,0.610292,0.823151,0.691651,0.688333,0.648255,0.761458,NaN,5.2565,3.6305,4.473682,3.496083
s144,0.375938,0.431437,0.3614,0.414586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.474013,0.44872,0.499418,0.500413,0.506333,NaN,NaN,NaN,2.115,1.6845,1.85575,1.591083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s646,0.580063,0.573906,0.529,0.555414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.729814,0.657167,0.748844,0.745492,0.814194,NaN,NaN,NaN,5.1665,2.4855,2.921542,1.931125
s647,0.747645,0.669156,0.628343,0.669828,NaN,NaN,0.654643,0.794625,0.435792,0.563583,0.602245,0.56164,0.54212,0.660171,0.322000,0.524608,0.487087,0.728089,0.660245,0.728794,0.5055,0.578937,NaN,3.1605,2.89,1.194667,1.37525
s648,0.549719,0.588,0.520971,0.522517,1.070545,1.142526,0.6765,0.793625,0.530333,0.501364,0.606327,0.615697,0.75,0.919797,0.571200,0.837587,0.685273,1.016537,0.936076,1.05562,0.708083,0.769312,NaN,4.983,6.057,4.835261,4.502826
s649,0.548438,0.563625,0.530229,0.512552,NaN,0.722359,0.510795,0.566583,0.38845,0.513391,0.775818,0.773444,0.651952,0.870422,0.576000,0.735153,0.61736,0.924815,0.819359,0.89558,0.709562,0.757042,NaN,4.175,2.761,3.124417,2.646083


In [27]:
FINAL_task_rt_df.style.apply(outlier_highlight, axis=None).to_excel("aim1_raw_rt.xlsx")

# Checking Task Acc / Errors

In [28]:
acc_cond_dict = {
    'twoByTwo': alldata_dict['twoByTwo']['switch_type'].unique().tolist(),
    'stopSignal': ['go', 'stop_failure'],
    'motorSelectiveStop': ['noncrit_signal', 'noncrit_nosignal', 'crit_stop_failure', 'crit_go'],
    'ANT': alldata_dict['ANT']['flanker_type'].unique().tolist(),
    'discountFix': ['smaller_sooner', 'larger_later'],
    'stroop': alldata_dict['stroop']['trial_type'].unique().tolist(),
    'WATT': None,
    'CCTHot': None
}

column_dict = {
    'twoByTwo': 'switch_type',
    'ANT': 'flanker_type', 
}

acc_funcs = {
    'CCTHot': CCTHot_EV_acc,
    'discountFix': discount_acc,
    'WATT': WATT_acc,
    'surveyMedley': surveyMedley_acc,
}

In [29]:
full_acc_dict = {}

for task in tasks:
    if task in acc_cond_dict:
        conditions = acc_cond_dict[task]
        cond_col = column_dict.get(task, 'trial_type')
    else:
        try:
            conditions = alldata_dict[task]['trial_type'].unique().tolist()
            cond_col = 'trial_type'
        except KeyError:
            print(f"Skipping {task} due to missing 'trial_type' column.")
            continue
                
    if task in acc_funcs:
        func = acc_funcs[task]
    else:
        func = acc_funcs.get(task, lambda x: acc_summary(x, conditions, condition_column = cond_col))
    
    full_acc_dict[task] = func(alldata_dict[task])
    
FINAL_task_acc_df = format_data(full_acc_dict, rt_flag = False, acc_flag = True, overall_metrics = True)

In [30]:
FINAL_task_acc_df.style.apply(outlier_highlight, axis=None).to_excel("aim1_raw_acc.xlsx")

## Outlier Analysis

In [31]:
data, non_truncated_subj = outlier_analysis(FINAL_task_acc_df, alldata_dict)

subject: s635 had a nonresponse rate of 0.6953125 for ANT
subject: s635 had a nonresponse rate of 0.5166666666666667 for twoByTwo
subject: s650 had a nonresponse rate of 0.68 for motorSelectiveStop
subject: s650 had nonresponses truncated for discountFix: 0.475
subject: s650 had a nonresponse rate of 0.5125 for twoByTwo
subject: s650 had nonresponses truncated for ANT: 0.328125
failed to truncate omission_rate was less than half of the df: 0.4666666666666667
subject: s650 had nonresponses truncated for stopSignal: 0.36
subject: s650 had a nonresponse rate of 0.625 for DPX
subject: s524 had nonresponses truncated for discountFix: 0.3
subject: s524 had nonresponses truncated for stroop: 0.17708333333333334
failed to truncate omission_rate was less than half of the df: 0.23076923076923078
subject: s613 had nonresponses truncated for discountFix: 0.21666666666666667
failed to truncate omission_rate was less than half of the df: 0.2857142857142857
subject: s613 had a nonresponse rate of 0.6

In [32]:
outlier_acc_dict = {}
for task in data.keys():

    if task in acc_cond_dict:
        conditions = acc_cond_dict[task]
        cond_col = column_dict.get(task, 'trial_type')
    else:
        try:
            conditions = data[task]['trial_type'].unique().tolist()
            cond_col = 'trial_type'
        except KeyError:
            print(f"Skipping {task} due to missing 'trial_type' column.")
            continue
                
    if task in acc_funcs:
        func = acc_funcs[task]
    else:
        func = acc_funcs.get(task, lambda x: acc_summary(x, conditions, condition_column = cond_col))
    
    outlier_acc_dict[task] = func(data[task])

outlier_adjusted_aim1_acc = format_data(outlier_acc_dict, rt_flag = False, acc_flag = True, overall_metrics = True)

In [33]:
df, dropped_subjects = overall_analysis(outlier_adjusted_aim1_acc, non_truncated_subj)

s519 dropped for having a noncrit signal omission rate of 0.6326530612244898
s549 dropped for having a noncrit signal omission rate of 0.42857142857142855
s577 dropped for having a noncrit signal omission rate of 0.5918367346938775
s650 dropped for having a noncrit signal omission rate of 0.7551020408163265
s548 dropped for having a stop success of 0.8
s557 dropped for having a stop success of 0.82
s574 dropped for having a stop success of 0.24
s577 dropped for having a stop success of 0.8
s608 dropped for having a stop success of 0.14
s650 dropped for having a stop success of 0.84
s512 dropped for having a stop success of 0.94
s556 dropped for having a stop success of 0.86
s585 dropped for having a stop success of 0.8


In [35]:
df.style.apply(outlier_highlight, axis=None).to_excel("outlier_adjusted_aim1_acc_new_events.xlsx")

## Dropped Subjects

In [36]:
# obtains df of exclusions for subjects_tasks for jeanette
exclusion_criteria = []
index = []
for sub in dropped_subjects.keys():
    for task in dropped_subjects[sub].keys():
        index.append(f'{sub}_{task}')
        for flag in dropped_subjects[sub][task]:
            exclusion_criteria.append(flag)

        
columns = set(exclusion_criteria)
exclusion_df = pd.DataFrame(np.nan, index = index, columns = columns)

for sub in dropped_subjects.keys():
    for task in dropped_subjects[sub].keys():
        exclusion_df.loc[f'{sub}_{task}', dropped_subjects[sub][task]] = 1

        
exclusion_df = exclusion_df.fillna(0)

#### Incorporating Jeanettes MRIQC Exclusions

In [37]:
mriqc_exclusions = pd.read_csv('QA_lev2_notes-mriqc.csv')

In [38]:
mriqc_exclusions = mriqc_exclusions.drop(columns=['session', 'notes', 'okay', 'fmriprep check (only done on questionable data)', 'SJ', 'Unnamed: 8'])

In [39]:
mriqc_exclusions = mriqc_exclusions[mriqc_exclusions['good_on_final_check'] == 'no']

In [40]:
updated_dropped_subjects = copy.deepcopy(dropped_subjects)

for ind in mriqc_exclusions.index:
    subject = mriqc_exclusions['subject'][ind]
    subject = f's{subject}'
    task = mriqc_exclusions['task'][ind]
    if task == 'WATT3':
        task = 'WATT'
        
    if subject in updated_dropped_subjects.keys():
        if task in updated_dropped_subjects[subject].keys():
            updated_dropped_subjects[subject][task].append('MRIQC_fail')
        else:
            updated_dropped_subjects[subject][task] = ['MRIQC_fail']
    else:
        updated_dropped_subjects[subject] = {task: ['MRIQC_fail']}
            

In [41]:
for sub in updated_dropped_subjects.keys():
    if len(updated_dropped_subjects[sub].keys()) >= 5:
        for task in tasks:
            if task in updated_dropped_subjects[sub].keys():
                if 'missing_more_than_half_the_tasks' not in updated_dropped_subjects[sub][task]:
                    updated_dropped_subjects[sub][task].append('missing_more_than_half_the_tasks')
            else:
                updated_dropped_subjects[sub][task] = ['missing_more_than_half_the_tasks']

In [42]:
# obtains df of exclusions for subjects_tasks for jeanette
exclusion_criteria = []
index = []
for sub in updated_dropped_subjects.keys():
    for task in updated_dropped_subjects[sub].keys():
        index.append(f'{sub}_{task}')
        for flag in updated_dropped_subjects[sub][task]:
            exclusion_criteria.append(flag)

        
columns = set(exclusion_criteria)
updated_exclusion_df = pd.DataFrame(np.nan, index = index, columns = columns)

for sub in updated_dropped_subjects.keys():
    for task in updated_dropped_subjects[sub].keys():
        updated_exclusion_df.loc[f'{sub}_{task}', updated_dropped_subjects[sub][task]] = 1

        
updated_exclusion_df = updated_exclusion_df.fillna(0)

In [43]:
updated_exclusion_df.to_csv("aim1_beh_subject_exclusions_new.csv")

##  Drop Metrics

In [44]:
tasks = ['ANT', 'discountFix', 'DPX', 'motorSelectiveStop', 'stopSignal', \
         'twoByTwo', 'stroop', 'WATT', 'CCTHot', 'surveyMedley']

In [45]:
metrics = {}

subjects = list(df.index.values)
subjects.remove('overall_mean')
subjects.remove('overall_std')

# creates a dictonary of subjects and all tasks
for subject in subjects:
    metrics[subject] = tasks    

In [46]:
# iterates through the dropped subjects dict and removes the dropped tasks from metrics
for subject in updated_dropped_subjects.keys():
    for task in updated_dropped_subjects[subject].keys():
        lst = metrics[subject]
        lst = list(filter(lambda x: x != task, lst))
        metrics.update({subject: lst})

In [47]:
# obtains the number of complete datasets
first_element_of_non_empty = [l[0] for l in metrics.values() if l]

num_non_empty = len(first_element_of_non_empty)
num_empty = len(metrics) - num_non_empty
print("Number of empty datasets: ", num_empty)
print("Number of non-datasets arrays: ", num_non_empty)

Number of empty datasets:  19
Number of non-datasets arrays:  91


In [48]:
complete_datasets = [l for l in metrics.values() if set(l) == set(tasks)]
num_complete = len(complete_datasets)
num_non_complete = len(metrics) - num_complete
num_partial = num_non_complete - num_empty
print("Number of complete datasets: ", num_complete)
print("Number of non-complete datasets: ", num_non_complete)
print("Number of partial datasets: ", num_partial)

Number of complete datasets:  68
Number of non-complete datasets:  42
Number of partial datasets:  23


In [49]:
task_metrics = {}
for task in tasks:
    test_list = [task]
    res = {idx : sum(1 for i in j if i in test_list) for idx, j in metrics.items()}
    
    task_occurence = sum(res.values())
    total = len(res)

    task_metrics[task] = {'percentage': task_occurence/total, 'dropped': total - task_occurence}
    
for task in task_metrics.keys():
    print(f'Number of dropped subjects for {task}: ', task_metrics[task]['dropped'])
    print(f'Total percentage kept for {task}: ', task_metrics[task]['percentage'])

Number of dropped subjects for ANT:  23
Total percentage kept for ANT:  0.7909090909090909
Number of dropped subjects for discountFix:  22
Total percentage kept for discountFix:  0.8
Number of dropped subjects for DPX:  23
Total percentage kept for DPX:  0.7909090909090909
Number of dropped subjects for motorSelectiveStop:  29
Total percentage kept for motorSelectiveStop:  0.7363636363636363
Number of dropped subjects for stopSignal:  22
Total percentage kept for stopSignal:  0.8
Number of dropped subjects for twoByTwo:  22
Total percentage kept for twoByTwo:  0.8
Number of dropped subjects for stroop:  20
Total percentage kept for stroop:  0.8181818181818182
Number of dropped subjects for WATT:  21
Total percentage kept for WATT:  0.8090909090909091
Number of dropped subjects for CCTHot:  20
Total percentage kept for CCTHot:  0.8181818181818182
Number of dropped subjects for surveyMedley:  19
Total percentage kept for surveyMedley:  0.8272727272727273


In [50]:
dataset_metrics = {
    'completed datasets': [num_complete],
    'fully dropped datasets': [num_empty],
    'partial datasets': [num_partial],
    'total': [total],
}

for task in tasks:
    dataset_metrics.update({f'{task} dropped': [task_metrics[task]['dropped']]})
    dataset_metrics.update({f'{task} kept': [total - task_metrics[task]['dropped']]})
    dataset_metrics.update({f'{task} percentage kept': [task_metrics[task]['percentage']]})

In [51]:
dataset_info = pd.DataFrame.from_dict(dataset_metrics)
dataset_info.rename(index={0:'aim1 beh data'},inplace=True)
dataset_info.T

,aim1 beh data
completed datasets,68.000000
fully dropped datasets,19.000000
partial datasets,23.000000
total,110.000000
ANT dropped,23.000000
ANT kept,87.000000
ANT percentage kept,0.790909
discountFix dropped,22.000000
discountFix kept,88.000000
discountFix percentage kept,0.800000


In [52]:
dataset_info.T.to_csv("aim1_dataset_metrics_new.csv")

## Pickling Subject List

In [ ]:
# dropped_subjects dictionary
with open('/home/groups/russpold/uh2_analysis/jaime_wonderland/Aim1_Neuro/Individual Neuro Analysis/aim1_dropped_dict.pickle', 'wb') as fh:
   pickle.dump(dropped_subjects, fh)

In [ ]:
# dropped_subjects dictionary
with open('/home/groups/russpold/uh2_analysis/jaime_wonderland/Aim1_Neuro/Individual Neuro Analysis/aim1_metrics.pickle', 'wb') as fh:
   pickle.dump(metrics, fh)

### Switching mapping for s644 Stroop task

In [ ]:
# Obtaining df
file = '/oak/stanford/groups/russpold/data/uh2/aim1/behavioral_data/processed/s644_stroop_cleaned.csv'
df = pd.read_csv(file)

# only looking at testing phase
subj_df =  df.loc[(df.loc[:,'exp_stage'] == 'test')]

# swapping mapping
subj_df.loc[subj_df['key_press'] == 71, 'key_press'] = 43
subj_df.loc[subj_df['key_press'] == 82, 'key_press'] = 71
subj_df.loc[subj_df['key_press'] == 43, 'key_press'] = 82

# adjusting correct columns to reflect
correct = (subj_df['key_press'] == subj_df['correct_response'])
subj_df['correct'][correct] = 1
incorrect = (subj_df['key_press'] != subj_df['correct_response'])
subj_df['correct'][incorrect] = 0

# calculating new accuracy
response_df = subj_df[subj_df.key_press != -1]
congruent = response_df[response_df.condition == 'congruent']
inc = response_df[response_df.condition == 'incongruent']
inc_acc = sum(inc['correct']) / len(inc['correct']) 
con_acc = sum(congruent['correct']) / len(congruent['correct']) 
print(f'con acc: {con_acc} // incon acc: {inc_acc}')

## Combining beh and brain qc for aim 1

In [ ]:
data_dir = '/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output'
tasks = ['ANT', 'discountFix', 'DPX', 'motorSelectiveStop', 'stopSignal', \
         'twoByTwo', 'stroop', 'WATT3', 'CCTHot']

In [ ]:
files = {}
for task in tasks:
    if task in ['WATT3', 'CCTHot']:
        files[task] = glob(path.join(data_dir, f'{task}_lev1_output', f'task_{task}_rtmodel_no_rt', f'excluded_subject.csv'))
    else:
        files[task] = glob(path.join(data_dir, f'{task}_lev1_output', f'task_{task}_rtmodel_rt_centered', f'excluded_subject.csv'))

In [ ]:
files

{'ANT': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/ANT_lev1_output/task_ANT_rtmodel_rt_centered/excluded_subject.csv'],
 'discountFix': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/discountFix_lev1_output/task_discountFix_rtmodel_rt_centered/excluded_subject.csv'],
 'DPX': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/DPX_lev1_output/task_DPX_rtmodel_rt_centered/excluded_subject.csv'],
 'motorSelectiveStop': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/motorSelectiveStop_lev1_output/task_motorSelectiveStop_rtmodel_rt_centered/excluded_subject.csv'],
 'stopSignal': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/stopSignal_lev1_output/task_stopSignal_rtmodel_rt_centered/excluded_subject.csv'],
 'twoByTwo': ['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/twoByTwo_lev1_output/task_twoByTwo_rtmodel_rt_centered/excluded_subject.csv'],
 'stroop': ['

In [ ]:
files['ANT']

['/oak/stanford/groups/russpold/data/uh2/aim1/BIDS/derivatives/output/ANT_lev1_output/task_ANT_rtmodel_rt_centered/excluded_subject.csv']

In [ ]:
exclusions = pd.read_csv(files['ANT'][0])

In [ ]:
exclusions

,subid_task,truncated_more_than_half_data,poor_performance_subjective_rating,greater_than_50_percent_nonresponse_rate,more_than_75_percent_stop_sucess_rate,less_than_25_percent_stop_sucess_rate,missing_beh_data,missing_more_than_half_the_tasks,percent_junk_gt_45,percent_scrub_gt_20,task_related_regressor_all_zeros,event_file_missing,confounds_file_missing,mask_file_missing,data_file_missing
0,623_ANT,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,061_ANT,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
2,607_ANT,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
3,609_ANT,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
4,599_ANT,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
5,600_ANT,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
6,644_ANT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,1.0,1.0
7,592_ANT,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0
8,635_ANT,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.726562,0.226714,0.0,0.0,0.0,0.0,0.0
9,526_ANT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.420664,0.0,0.0,0.0,0.0,0.0
